In [45]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import os
import pickle

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [42]:
def load_data(filename):
    """load data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding = 'latin')
        return data['data'], data['labels']
    
class CifarData:
    """定义一个类处理cifar的数据"""
    def __init__(self, filenames, need_shuffle): #对于训练集来说需要shuffle，打乱数据以增加训练成效，对于测试集来说就不需要。
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename) 
            '''this is a 二分类器，so we only need 0 and 1 in files, so we need to make a filter.'''
            for item,label in zip(data,labels):
                if label in [0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data) #纵向组合成一个矩阵
        self._data = self._data / 127.5 - 1   
        self._labels = np.hstack(all_labels) #横向组合成一个矩阵
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
                
    def _shuffle_data(self):
        #[0,1,2,3,4,5] ->[3,4,2,5,1.0] 在训练集上做
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data , batch_labels

    
    '''至此处理完了cifar数据'''


train_filemnames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_data_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filemnames,True)
test_data = CifarData(test_data_filenames,False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)
[[-0.08235294  0.03529412 -0.00392157 ...  0.24705882  0.25490196
   0.2627451 ]
 [ 0.52941176  0.56078431  0.59215686 ... -0.03529412 -0.06666667
  -0.01960784]
 [-0.51372549 -0.50588235 -0.51372549 ... -0.36470588 -0.41176471
  -0.52156863]
 ...
 [ 0.48235294  0.52156863  0.52941176 ...  0.1372549   0.17647059
   0.14509804]
 [ 0.21568627  0.20784314  0.20784314 ... -0.18431373 -0.18431373
  -0.17647059]
 [ 0.44313725  0.23137255  0.25490196 ...  0.3254902   0.30980392
   0.24705882]]
[1 0 1 1 0 1 0 0 0 1]


In [22]:
"""在tensorflow里，要先把图搭建起来，再往里面填数据。"""
tf.reset_default_graph()
x = tf.placeholder(tf.float32,[None,3072])  
#这是一个占位符，也是一个变量，可以往里面塞数据。这是data的palceholder，None表示样本数是不确定的。

y = tf.placeholder(tf.int64,[None])  
#这是labels的placeholder，因为都是离散变量，所以用int64

#（3072，1）
w = tf.get_variable('w',[x.get_shape()[-1],1],
                   initializer = tf.random_normal_initializer(0,1)) 
#w是要和x做内积的权重，w的大小就是x的大小中的3072，这是一个二分类器，所以输出为1，
#还需要定义一个初始化的方法，这里使用比较通用的正态分布。均值是1，方差为1.


#（1，）                                                                    
b = tf.get_variable('b',[1],
                   initializer =tf.constant_initializer(0.0) )  
#b是偏置，大小为w的第二维度，也就是1，输出也是1，初始化方法使用常数初始化0.

#[None,3072]*[3072,1] = [None,1]
y_ = tf.matmul(x,w) + b # y_是x和w做内积加上偏置b后的结果。因为x和w在多样本下都是矩阵，所以用matmul矩阵乘法

#[None,1]
p_y_1 = tf.nn.sigmoid(y_)  #把y_转换成概率值，就是放到sigmoid函数中。这是y=1的概率

'''为了把y和p_y_1和y做差别，所以要让他们的shape和数值单位一致，所以要把y进行reshape'''
y_reshaped = tf.reshape(y,(-1,1))
y_reshaped_float = tf.cast(y_reshaped,tf.float32)

loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))  #损失函数的表达式，差的平方和的均值。


"""我们还对准确率感兴趣，所以现在来计算准确率"""
#bool
predict = p_y_1 > 0.5  #以这个概率大于百分之五十为正确，记为1.

#[1,0,1,1,1,0,1]类似的
correct_prediction = tf.equal(tf.cast(predict,tf.int64),y_reshaped) 
#这里指的是我们的模型预测出的值是否正确的布尔值转换成int后的值和y_reshaped是否相等的布尔值

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64)) #准确率就是我们上面所说的这个值的平均值

'''定义一个梯度下降的方法'''
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) 
    #AdamOptimizer是梯度下降的一个变种，1e-3是一个初始化的generate，并且我们优化的是loss值


In [44]:
init = tf.global_variables_initializer() 
# 构建一个初始化函数，用于初始化数据
batch_size = 20
train_steps = 50000
test_steps = 100
'''构建一个session，用于执行计算图'''
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss:%4.5f, acc:%4.5f'\
                 %(i+1,loss_val,acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_data_filenames, False)
            all_test_acc_val =[]
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Steps: %d, acc:%4.5f' %(i+1, test_acc))
#通过调用session的run函数执行计算图。可以计算loss, accuracy, train_op。加了train_op就是在训练，没加就只是在测试。
#feed_dict是我们输入的数据，x是cifar10的图片数据，y是label数据


[Train] Step: 500, loss:0.19298, acc:0.80000
[Train] Step: 1000, loss:0.22279, acc:0.75000
[Train] Step: 1500, loss:0.15425, acc:0.85000
[Train] Step: 2000, loss:0.30007, acc:0.70000
[Train] Step: 2500, loss:0.22056, acc:0.75000
[Train] Step: 3000, loss:0.21085, acc:0.75000
[Train] Step: 3500, loss:0.30349, acc:0.70000
[Train] Step: 4000, loss:0.14557, acc:0.85000
[Train] Step: 4500, loss:0.29734, acc:0.70000
[Train] Step: 5000, loss:0.17265, acc:0.80000
(2000, 3072)
(2000,)
[Test] Steps: 5000, acc:0.80650
[Train] Step: 5500, loss:0.04424, acc:0.95000
[Train] Step: 6000, loss:0.06072, acc:0.95000
[Train] Step: 6500, loss:0.25406, acc:0.70000
[Train] Step: 7000, loss:0.19669, acc:0.80000
[Train] Step: 7500, loss:0.09957, acc:0.90000
[Train] Step: 8000, loss:0.05000, acc:0.95000
[Train] Step: 8500, loss:0.19999, acc:0.80000
[Train] Step: 9000, loss:0.12714, acc:0.85000
[Train] Step: 9500, loss:0.15000, acc:0.85000
[Train] Step: 10000, loss:0.15020, acc:0.85000
(2000, 3072)
(2000,)
[Test]